# Control allocation example for MilliAmpere1

## Quadratic Programming based Control Allocation

For the sake of testing, set `PYTHONPATH` so notebook can be run without installing the package.

In [1]:
import sys
sys.path.insert(0, "../src")

First, import the control allocation packge

In [2]:
import skadipy

import numpy as np

Now, we describe the thrusters. There are different options for thrusters. These are `Fixed`, `Aximuth`, `Vectored`.
It is defined by the position of the thruster and the direction that its directed at.
Orientation of the thruster is defined using quaternions.

Depending on the allocator choice, `extra_attributes` dictionary changes.
For milliAmpere1, we use Quadratic Programming allocator. It takes `limits` list to set thrust limits for the thrusters.

X axis is forward, Y axis is starboard, Z axis is down.
For simplicty we will assume that the ship is symmetric and that the thrusters are
symmetrically placed. The azimuthing thrusters are also fixed in the vertical plane.

In [3]:

ma_bow_port = skadipy.actuator.Fixed(
    position=skadipy.toolbox.Point([0.3, -0.1, 0.0]),
    orientation=skadipy.toolbox.Quaternion(
        axis=(0.0, 0.0, 1.0), angle=(3*np.pi / 4.0)),
    extra_attributes={
        "limits": [0.0, 1.0],
        "name": "bow_port"
    }
)
ma_bow_starboard = skadipy.actuator.Fixed(
    position=skadipy.toolbox.Point([0.3, 0.1, 0.0]),
    orientation=skadipy.toolbox.Quaternion(
        axis=(0.0, 0.0, 1.0), angle=(-3*np.pi / 4.0)),
    extra_attributes={
        "limits": [0.0, 1.0],
        "name": "bow_starboard"
    }
)
ma_aft_port = skadipy.actuator.Fixed(
    position=skadipy.toolbox.Point([-0.3, -0.1, 0.0]),
    orientation=skadipy.toolbox.Quaternion(
        axis=(0.0, 0.0, 1.0), angle=(np.pi / 4.0)),
    extra_attributes={
        "limits": [0.0, 1.0],
        "name": "aft_port"
    }
)
ma_aft_starboard = skadipy.actuator.Fixed(
    position=skadipy.toolbox.Point([-0.3, 0.1, 0.0]),
    orientation=skadipy.toolbox.Quaternion(
        axis=(0.0, 0.0, 1.0), angle=(-np.pi / 4.0)),
    extra_attributes={
        "limits": [0.0, 1.0],
        "name": "aft_starboard"
    }
)

# Put all actuators in a list and create the allocator object
actuators = [
    ma_bow_port,
    ma_bow_starboard,
    ma_aft_port,
    ma_aft_starboard

]

Now we need to describe what degrees of freedom we want to control.
To specify that, we use `ForceTorqueComponent` class.
Possible options are

- `ForceTorqueCompontent.X` for surge
- `ForceTorqueCompontent.Y` for sway
- `ForceTorqueCompontent.Z` for heave
- `ForceTorqueCompontent.K` for roll
- `ForceTorqueCompontent.M` for pitch
- `ForceTorqueCompontent.N` for yaw


In [4]:
# List all the degrees of freedom that we want to control
dofs = [
    skadipy.allocator._base.ForceTorqueComponent.X,
    skadipy.allocator._base.ForceTorqueComponent.Y,
    skadipy.allocator._base.ForceTorqueComponent.N
]

Next, we create the allocator object using the thrusters and the degrees of freedom we want to control.

In [5]:
# Create the allocator object
allocator = skadipy.allocator.QuadraticProgramming(
    actuators=actuators,
    force_torque_components=dofs
)

To allow change in vehicle configuration, allocation matrix can be updated using `update_allocation_matrix` method.

Finally, we can use the allocator to allocate the forces and torques to the thrusters.

In [6]:
# Compute or update the configuration matrix
allocator.compute_configuration_matrix()

Regardless of the DOF we want to control, the allocator will always return forces and torques in the body frame.
If we want to control the vehicle in the NED frame, we need to rotate the forces and torques to the NED frame.

The $\tau_{\text{cmd}}$ **must** be in the form of

$$
\tau_{\text{cmd}} = \begin{bmatrix} F_x & F_y & F_z & M_x & M_y & M_z \end{bmatrix}
$$

where $F_x, F_y, F_z$ are the forces in the body frame and $M_x, M_y, M_z$ are the torques in the body frame.

In [7]:
tau_cmd = np.array(
    [[1],
     [0],
     [0],
     [0],
     [0],
     [1]], dtype=np.float32)

# Allocate a control signal
allocator.allocate(tau=tau_cmd)

# Get the allocated force
print(f"Allocated {allocator.allocated}")

Allocated [ 0.82251267 -0.00700104  0.          0.          0.          0.11833995]


To get commanded forces to the thrusters, we can use the actuator objects.

In [8]:
for actuator in actuators:
    print(f"{actuator.extra_attributes['name']}: {actuator.force}")

bow_port: [0.41344483]
bow_starboard: [4.07849815e-07]
aft_port: [0.57665421]
aft_starboard: [0.99999961]
